In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import time
from threading import Thread

# Data consent information
data_consent_msg = widgets.Label(value="""DATA CONSENT INFORMATION:\n
Please read:\n
We wish to record your response data to an anonymised public data repository.\n
Your data will be used for educational teaching purposes practising data analysis and visualisation.\n
Please type 'yes' in the box below if you consent to the upload.""")
data_consent_msg.layout.width = '200%'

data_consent_input = widgets.Text()
submit_consent_button = widgets.Button(description="Submit")

# Input for user ID defined at the top level
user_id_input = widgets.Text(description="Your ID:")

# Output area for displaying results
output_area = widgets.Output()

# Function for ID submission and showing the user form
def on_id_submit(b):
    with output_area:
        clear_output()
        user_id = user_id_input.value
        print("User entered ID:", user_id)  # This print statement can be replaced with any other logic needed
        # Now display the rest of the user form
        display(user_info_form, start_test_button)

# Function for consent submission
def on_consent_submit(b):
    if data_consent_input.value.lower() == "yes":
        with output_area:
            clear_output()
            # Instructions for the user to create an anonymised ID
            id_instructions = widgets.Label(value="""
Enter your anonymised ID
To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor/actress
e.g. if your friend was called Charlie Brown and the film star was Tom Cruise,
then your unique identifier would be CBTC
""")
            id_instructions.layout.width = '100%'
            
            # Button for submitting ID
            submit_id_button = widgets.Button(description="Submit ID")
            
            # Event handler for the ID submission button
            submit_id_button.on_click(on_id_submit)
            
            # Display the ID instructions, input, and submit button
            display(id_instructions, user_id_input, submit_id_button)
    else:
        with output_area:
            clear_output()
            print("You have not consented to provide data. The test cannot be started.")

submit_consent_button.on_click(on_consent_submit)

# User information input form
user_info_form = widgets.VBox([
    widgets.IntText(description="Age:"),
    widgets.Dropdown(description="Gender:", options=["Male", "Female", "Other"]),
    widgets.IntText(description="Hours of sleep:"),
    widgets.Dropdown(description="Caffeine in 6 hours:", options=["Yes", "No"]),
    widgets.Dropdown(description="Chronotype:", options=["Evening", "Morning"]),
    widgets.IntText(description="Test time (hour):"),
    widgets.IntSlider(description="How sharp are you feeling mentally at the moment?", min=1, max=10, value=5)
])

# Button to start the test
start_test_button = widgets.Button(description="Start the Math Test")

# Display the widgets on the screen
display(data_consent_msg, data_consent_input, submit_consent_button, output_area)

# ... include the rest of your test logic here, including the `start_test` function ...
# Function to generate a math question
def generate_question(level):
    operation_dict = {0: '+', 1: '-', 2: '*', 3: '/'}
    if level == 1:  # Basic arithmetic with single digits
        num1, num2 = random.randint(1, 9), random.randint(1, 9)
        operation = operation_dict[random.randint(0, 3)]
    elif level == 2:  # Basic arithmetic with double digits
        num1, num2 = random.randint(10, 99), random.randint(10, 99)
        operation = operation_dict[random.randint(0, 3)]
    elif level == 3:  # Include square, cube, and root
        num1, num2 = random.randint(1, 9), random.randint(1, 9)
        extra_ops = ['**2', '**3', '**0.5']
        operation = random.choice(list(operation_dict.values()) + extra_ops)
    elif level >= 4:  # Multiple operations
        num1, num2, num3 = random.randint(1, 9), random.randint(1, 9), random.randint(1, 9)
        operation1 = operation_dict[random.randint(0, 3)]
        operation2 = operation_dict[random.randint(0, 3)]
        return f"{num1} {operation1} {num2} {operation2} {num3}"

    if operation in ['/', '**0.5'] and num2 == 0:  # Prevent division by zero or sqrt of zero
        num2 = random.randint(1, 9)

    return f"{num1} {operation} {num2}"

# Function to handle the start of the test
def start_test(b):
    with output_area:
        clear_output(wait=True)

    level = 1  # Initial difficulty level
    question_count = 0
    correct_answers = 0
    test_duration = 180  # Test duration in seconds (3 minutes)

    # Initialize widgets for the test
    question_label = widgets.Label()
    answer_box = widgets.Text()
    submit_button = widgets.Button(description="Submit Answer")
    result_label = widgets.Label()

    # Function to check answer and move to next question
    def check_answer(b):
        nonlocal correct_answers, question_count, level  # Declare nonlocal for the enclosed variables
        user_answer = answer_box.value
        try:
            if eval(question_label.value[:-4]) == float(user_answer):
                correct_answers += 1
                result_label.value = "Correct!"
            else:
                result_label.value = "Incorrect. Right answer was: " + str(eval(question_label.value[:-4]))
        except Exception as e:
            result_label.value = "Error in calculation."
        finally:
            next_question()

    submit_button.on_click(check_answer)

    # Function to display next question
    def next_question():
        nonlocal level, question_count  # Declare nonlocal for the enclosed variables
        if question_count < 10:  # Assuming a maximum of 10 questions
            question = generate_question(level)
            question_label.value = question + " = ?"
            answer_box.value = ""
            question_count += 1
            if question_count % 3 == 0 and level < 4:  # Adjust this condition as per your logic
                level += 1
            with output_area:
                clear_output(wait=True) 
                display(question_label, answer_box, submit_button, result_label)
        else:
            # If question count has reached the limit, end the test
            test_timer()

    # Start the test with the first question
    next_question()

    # Timer to end test after 3 minutes
    def test_timer():
        nonlocal question_count, correct_answers  # Declare nonlocal for the enclosed variables
        time.sleep(test_duration)
        with output_area:
            clear_output()
            final_message = f"Test completed! You answered {correct_answers} out of {question_count} questions correctly."
            print(final_message)

    timer_thread = Thread(target=test_timer)
    timer_thread.start()

# Assigning the start test function to the button
start_test_button.on_click(start_test)


Label(value="DATA CONSENT INFORMATION:\n\nPlease read:\n\nWe wish to record your response data to an anonymise…

Text(value='')

Button(description='Submit', style=ButtonStyle())

Output()